In [25]:
# 必要的导入
import pickle
import os
import json
from pathlib import Path
import pandas as pd
from glob import glob
import numpy as np
import backtrader as bt
import backtrader.analyzers as btanalyzers
import optuna
import time
from datetime import datetime, timedelta
import concurrent.futures
import matplotlib.pyplot as plt
import math
from typing import Dict, List, Tuple, Any, Optional
import threading
from queue import Queue
import warnings

# 忽略警告
warnings.filterwarnings('ignore')

# 全局缓存
_data_feed_cache = {}
_data_completeness_cache = {}

## 1. 添加配置单元格

In [26]:
# 导入 RSIPAPTP 策略
from RSIPAPTP import RSIPAPTP

# 全局配置
CONFIG = {
    # 策略相关配置
    'strategy': {
        'class': RSIPAPTP,
        'name': RSIPAPTP.__name__
    },
    
    # 如果 selected_symbols 为空，则通过 get_all_symbols 自动获取所有交易对
    # 'selected_symbols': [
    #     "BTCUSDT",
    #     "ETHUSDT",
    #     "SOLUSDT",
    #     "XRPUSDT",
    #     "DOGEUSDT",
    #     "BNBUSDT",
    #     "NEARUSDT",
    #     "ADAUSDT",
    #     "EOSUSDT",
    #     "LTCUSDT",
    #     "SUIUSDT",
    #     "1000PEPEUSDT",
    #     "AVAXUSDT",
    #     "LINKUSDT",
    #     "AAVEUSDT",
    #     "TRXUSDT",
    #     "ICPUSDT",
    #     "WLDUSDT",
    #     "DOTUSDT",
    #     "APTUSDT",
    #     "UNIUSDT",
    #     "FILUSDT",
    #     "RAYUSDT",
    #     "1000SHIBUSDT",
    #     "SEIUSDT",
    #     "ARBUSDT",
    #     "ATOMUSDT",
    #     "LDOUSDT",
    #     "INJUSDT",
    #     "OPUSDT"
    # ],
    'selected_symbols': [
        "AAVEUSDT",
        "1000PEPEUSDT",
        "APTUSDT",

    ],

    # 数据路径配置 - 按优先级排序，将自动选择第一个有效的路径
    'data_paths': [
        r'\\znas\Main\futures',  # 本地网络路径
        r'.\futures',            # 当前目录下的futures文件夹
        r'.\..\..\futures',      # 从BacktestsOptimization\RSIProgressiveTP向上两级到根目录的futures
        r'..\..\futures',        # 另一种表示方式
        r'.\..\..\..\..\futures', # 再往上一级
        r'..\futures',           # 原来的路径
        '../futures',            # 原来的路径（Unix风格）
    ],
    
    # 其他设置保持不变
    'data_path': '../futures',  # 默认数据路径（将被自动解析的路径替换）
    # 'data_path': r'\\znas\Main\futures',  # 使用原始字符串表示法
    'start_date': '2024-01-01',
    'end_date': '2025-04-03',
    'source_timeframe': '1m',
    'target_timeframes': ['1H'],  # 这里可以设置多个时间周期
    
    # 文件保存配置
    'reports_path': 'reports_walkforward',
    'equity_curves_dir': 'equity_curves_walkforward',
    'results_filename_template': 'walkforward_results_{strategy_name}_{start_date}-{end_date}.csv',
    
    # 回测参数配置
    'commission': 0.0004,
    'initial_capital': 10000,
    
    # 优化参数配置，与批量优化相同
    'optimization_params': {
        'port': range(5, 20, 1),                # 用于开仓的投资组合百分比，5%-20%
        'ps2': range(1, 5, 1),                # 第2层多头入场百分比，1%-5%
        'ps3': range(2, 8, 1),                # 第3层多头入场百分比，2%-8%
        'ps4': range(3, 10, 1),               # 第4层多头入场百分比，3%-10%
        'ps5': range(5, 15, 1),                 # 第5层多头入场百分比，5%-15%
        'ps6': range(10, 25, 1),                # 第6层多头入场百分比，10%-25%
        'ps7': range(15, 35, 1),                # 第7层多头入场百分比，15%-35%
        'ps8': range(25, 50, 2),                # 第8层多头入场百分比，25%-50%
        'ma_length': range(50, 200, 10),        # 移动平均线周期，50-200
        'rsi_length': range(7, 21, 1),          # RSI周期，7-21
        'oversold': range(20, 40, 1),           # RSI超卖阈值，20-40
        'profit_target_percent': range(1, 6, 1),      # 每个仓位止盈百分比，1%-6%
        'profit_target_percent_all': range(2, 8, 1),  # 总体止盈百分比，2%-8%
        'take_profit_progression': range(5, 20, 1),     # 止盈递进，5%-20%
        'entry_on': [True, False]               # 新入场是否影响止盈限制
    },
    
    # 优化通用设置
    'optimization_settings': {
        'min_trades': 10,       # 最少交易次数
        'timeout': 3600,        # 超时时间（秒）
        'n_trials': 200,        # 每个优化窗口的试验次数
        'n_jobs': 20            # optuna并行优化作业数
    },
    
    # Walk Forward特有设置
    'walkforward_settings': {
        'optimization_period_days': 84,  # 优化窗口长度（天）
        'out_of_sample_period_days': 14, # 样本外测试窗口长度（天）
        'n_threads': 20                  # 多交易对处理的线程数
    },
    
    # 性能设置
    'performance_settings': {
        'preload_data': True,            # 是否预加载数据
        'use_cache': True,               # 是否使用缓存
        'debug_mode': False              # 是否启用调试模式
    }
}

## 1. 数据加载函数

In [27]:
def get_timeframe_params(timeframe_str):
    """
    将时间周期字符串转换为 backtrader 的 timeframe 和 compression 参数
    """
    if timeframe_str.endswith('min'):
        return (bt.TimeFrame.Minutes, int(timeframe_str.replace('min', '')))
    elif timeframe_str.endswith('h') or timeframe_str.endswith('H'):
        minutes = int(timeframe_str.replace('h', '').replace('H', '')) * 60
        return (bt.TimeFrame.Minutes, minutes)
    elif timeframe_str.endswith('D'):
        return (bt.TimeFrame.Days, 1)
    elif timeframe_str == '1m':
        return (bt.TimeFrame.Minutes, 1)
    else:
        raise ValueError(f"不支持的时间周期格式: {timeframe_str}")

def resolve_data_path():
    """
    检查CONFIG中配置的多个可能的数据路径，返回第一个存在的路径
    如果都不存在，则返回默认路径
    
    Returns:
        str: 有效的数据路径
    """
    # 从CONFIG中获取可能的路径列表
    base_paths = CONFIG.get('data_paths', ['../futures'])
    default_path = CONFIG.get('data_path', '../futures')
    
    # 尝试每个路径
    for path in base_paths:
        try:
            if os.path.exists(path):
                print(f"找到有效数据路径: {path}")
                return path
        except Exception as e:
            print(f"检查路径 {path} 时出错: {str(e)}")
    
    # 如果没有找到有效路径，返回默认路径
    print(f"未找到有效数据路径，使用默认路径: {default_path}")
    return default_path

def load_and_resample_data(symbol, start_date, end_date, source_timeframe='1m', target_timeframe='30min', data_path=None):
    """
    加载并重采样期货数据，并缓存已经重采样后的 DataFrame 以避免重复 I/O 操作
    """
    # 如果没有提供数据路径或路径无效，则使用解析后的路径
    if data_path is None or not os.path.exists(data_path):
        data_path = CONFIG.get('resolved_data_path', resolve_data_path())
    
    # 构造缓存键
    key = (symbol, start_date, end_date, source_timeframe, target_timeframe, data_path)
    if key in _data_feed_cache:
        # 如果缓存中有，返回新的数据馈送对象（注意拷贝，防止被修改）
        cached_df = _data_feed_cache[key]
        timeframe, compression = get_timeframe_params(target_timeframe)
        data_feed = bt.feeds.PandasData(
            dataname=cached_df.copy(),
            open='Open',
            high='High',
            low='Low',
            close='Close',
            volume='Volume',
            openinterest=-1,
            timeframe=timeframe,
            compression=compression,
            fromdate=pd.to_datetime(start_date),
            todate=pd.to_datetime(end_date)
        )
        
        # 添加clone方法，这样可以快速创建数据副本而不需要重新执行IO
        data_feed.clone = lambda: bt.feeds.PandasData(
            dataname=cached_df.copy(),
            open='Open',
            high='High',
            low='Low',
            close='Close',
            volume='Volume',
            openinterest=-1,
            timeframe=timeframe,
            compression=compression,
            fromdate=pd.to_datetime(start_date),
            todate=pd.to_datetime(end_date)
        )
        
        return data_feed
    
    # 生成日期范围
    date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    all_data = []
    
    # 标准化交易对名称
    formatted_symbol = symbol.replace('/', '_').replace(':', '_')
    if not formatted_symbol.endswith('USDT'):
        formatted_symbol = f"{formatted_symbol}USDT"
    
    # 顺序读取文件，不使用线程池
    for date in date_range:
        date_str = date.strftime('%Y-%m-%d')
        # 构建文件路径
        file_path = os.path.join(data_path, date_str, f"{date_str}_{formatted_symbol}_USDT_{source_timeframe}.csv")
        
        try:
            if os.path.exists(file_path):
                # 读取数据
                df = pd.read_csv(file_path)
                df['datetime'] = pd.to_datetime(df['datetime'])
                all_data.append(df)
            else:
                print(f"文件不存在: {file_path}")
        except Exception as e:
            print(f"读取文件出错 {file_path}: {str(e)}")
            continue
    
    if not all_data:
        raise ValueError(f"未找到 {symbol} 在指定日期范围内的数据")
    
    # 合并、排序，以及重采样数据
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df = combined_df.sort_values('datetime')
    combined_df.set_index('datetime', inplace=True)
    
    resampled = combined_df.resample(target_timeframe).agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }).dropna()  # 立即删除NaN值
    
    backtesting_df = pd.DataFrame({
        'Open': resampled['open'],
        'High': resampled['high'],
        'Low': resampled['low'],
        'Close': resampled['close'],
        'Volume': resampled['volume']
    })
    
    # 确保所有数据都是数值类型并删除任何无效值
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        backtesting_df[col] = pd.to_numeric(backtesting_df[col], errors='coerce')
    backtesting_df = backtesting_df.dropna()
    
    # 将结果缓存在全局变量中（使用拷贝，以免后续被修改）
    _data_feed_cache[key] = backtesting_df.copy()
    
    timeframe, compression = get_timeframe_params(target_timeframe)
    data_feed = bt.feeds.PandasData(
        dataname=backtesting_df,
        open='Open',
        high='High',
        low='Low',
        close='Close',
        volume='Volume',
        openinterest=-1,
        timeframe=timeframe,
        compression=compression,
        fromdate=pd.to_datetime(start_date),
        todate=pd.to_datetime(end_date)
    )
    
    # 添加clone方法
    data_feed.clone = lambda: bt.feeds.PandasData(
        dataname=backtesting_df.copy(),
        open='Open',
        high='High',
        low='Low',
        close='Close',
        volume='Volume',
        openinterest=-1,
        timeframe=timeframe,
        compression=compression,
        fromdate=pd.to_datetime(start_date),
        todate=pd.to_datetime(end_date)
    )
    
    return data_feed

## 2. 回测分析器类

In [28]:
class AcctStats(bt.Analyzer):
    """账户统计分析器"""
    
    def __init__(self):
        self.start_val = self.strategy.broker.get_value()
        self.end_val = None

    def stop(self):
        self.end_val = self.strategy.broker.get_value()

    def get_analysis(self):
        return {"start": self.start_val, "end": self.end_val}

class ValueStats(bt.Analyzer):
    """账户价值跟踪分析器"""
    
    def __init__(self):
        self.val = []

    def next(self): 
        self.val.append(self.strategy.broker.get_value())

    def get_analysis(self):
        return self.val

## 3. 时间框架映射表

## 5. 回测执行函数

## 6. Optuna优化相关函数

## 7. WalkForward核心函数

In [29]:


def run_backtest(symbol, timeframe, start_timestamp, end_timestamp, strategy_params, capital,
                 plot=False, print_log=False):
    """
    执行单次回测
    """
    try:
        # 加载数据 - 使用clone避免重复加载
        data_feed = load_and_resample_data(symbol, start_timestamp, end_timestamp, timeframe)
        
        # 创建cerebro实例
        cerebro = bt.Cerebro(
            optdatas=True,    
            optreturn=True,   
            runonce=True,     
            preload=True      
        )
        
        # 打印当前运行的策略参数（用于调试）
        if print_log:
            print(f"\n{'-'*20} 回测设置 {'-'*20}")
            print(f"交易对: {symbol} | 时间框架: {timeframe}")
            print(f"时间区间: {datetime.fromtimestamp(start_timestamp).strftime('%Y-%m-%d')} → {datetime.fromtimestamp(end_timestamp).strftime('%Y-%m-%d')}")
        
        # 从interval_params提取最后一组参数(适用于当前测试的参数)
        if 'interval_params' in strategy_params and strategy_params['interval_params']:
            # 获取最后一个时间段的参数
            last_params = strategy_params['interval_params'][-1][1]
            # 添加策略，直接将参数传递给策略，不传递printlog参数
            cerebro.addstrategy(CONFIG['strategy']['class'], **last_params)
            
            # 打印策略参数（用于调试）
            if print_log:
                print(f"\n策略参数:")
                for param_name, param_value in last_params.items():
                    print(f"  - {param_name}: {param_value}")
        else:
            # 没有区间参数时的默认处理
            cerebro.addstrategy(CONFIG['strategy']['class'])
            if print_log:
                print("使用默认策略参数")
        
        # 设置手续费和滑点
        if 'slippage' in strategy_params:
            cerebro.broker.set_slippage_perc(perc=strategy_params['slippage'])
        if 'commission' in strategy_params:
            cerebro.broker.setcommission(commission=strategy_params['commission'])
        
        # 添加分析器
        cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='TradeAnalysis')
        cerebro.addanalyzer(btanalyzers.SharpeRatio, timeframe=bt.TimeFrame.Days, 
                            riskfreerate=0.0, _name='SharpeAnalysis')
        cerebro.addanalyzer(btanalyzers.DrawDown, _name='DrawDownAnalysis')
        cerebro.addanalyzer(AcctStats, _name='ActualAnalysis')
        cerebro.addanalyzer(ValueStats, _name='ValueAnalysis')
        cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
        
        # 添加数据和设置初始资金
        cerebro.adddata(data_feed)
        cerebro.broker.setcash(capital)
        
        # 打印起始条件
        if print_log:
            print(f'Starting Portfolio Value: {cerebro.broker.getvalue():.2f}')
        
        # 运行回测
        results = cerebro.run()
        
        # 打印最终结果
        if print_log:
            print(f'Final Portfolio Value: {cerebro.broker.getvalue():.2f}')
        
        # 提取分析结果
        result = results[0]
        trade_analysis = result.analyzers.TradeAnalysis.get_analysis()
        sharpe_analysis = result.analyzers.SharpeAnalysis.get_analysis()
        drawdown_analysis = result.analyzers.DrawDownAnalysis.get_analysis()
        value_analysis = result.analyzers.ValueAnalysis.get_analysis()
        # 正确获取Returns analyzer的结果
        returns_analysis = result.analyzers.returns.get_analysis()
        
        # 直接计算总收益率（转为百分比）
        total_return = returns_analysis.get('rtot', 0) * 100
        
        # 处理其他指标
        num_won_trades = trade_analysis['won']['total'] if 'won' in trade_analysis else 0
        num_lost_trades = trade_analysis['lost']['total'] if 'lost' in trade_analysis else 0
        win_ratio = num_won_trades / (num_won_trades + num_lost_trades) if (num_won_trades + num_lost_trades) > 0 else 0
        sharpe_ratio = sharpe_analysis.get('sharperatio', 0)
        
        # 使用total_return代替原有的盈亏百分比计算
        percentage_pnl = total_return
        max_drawdown = drawdown_analysis['max']['drawdown'] if 'max' in drawdown_analysis else 0
        calmar_ratio = percentage_pnl / math.sqrt(1 + max_drawdown) if max_drawdown > 0 else 0
        
        # 如果要绘图
        if plot:
            cerebro.plot(style='candle')
            
        return {
            'win_ratio': win_ratio,
            'percentage_pnl': percentage_pnl,
            'sharpe_ratio': sharpe_ratio,
            'calmar_ratio': calmar_ratio,
            'trade_analysis': trade_analysis,
            'value_analysis': value_analysis,
            'total_return': total_return  # 添加计算好的总收益率字段
        }
        
    except Exception as e:
        print(f"回测执行错误: {str(e)}")
        return {
            'win_ratio': 0.0,
            'percentage_pnl': 0.0,
            'sharpe_ratio': 0.0,
            'calmar_ratio': 0.0,
            'trade_analysis': {},
            'value_analysis': [],
            'total_return': 0.0,  # 添加默认的总收益率
            'error': str(e)
        }

## 8. 样本外测试函数

## 9. 多交易对批处理与结果汇总

In [30]:
def evaluate_parameters(data, params, capital):
    """
    在样本外区间评估优化的参数
    
    Args:
        data: 已经加载好的回测数据，包含正确的日期范围
        params: 策略参数
        capital: 初始资金
    """
 
 
    
    print(f"在样本外区间评估参数...")
    
    # 创建回测引擎
    cerebro = bt.Cerebro(
                optdatas=True,
                optreturn=True,
                runonce=True,
                preload=True
            )
    
    # 添加数据
    cerebro.adddata(data.clone())
    
    # 添加策略
    cerebro.addstrategy(CONFIG['strategy']['class'], **params)
    
    # 设置初始资金
    cerebro.broker.setcash(capital)
    
    # 设置手续费
    cerebro.broker.setcommission(commission=CONFIG['commission'])
    
    # 添加分析器
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    
    # 运行回测
    strategies = cerebro.run()
    strat = strategies[0]
    
    # 获取最终价值
    final_value = cerebro.broker.getvalue()
    
    # 获取交易统计
    trades = strat.analyzers.trades.get_analysis()
    
    # 获取收益率统计
    returns_stats = strat.analyzers.returns.get_analysis()
    
    # 获取PyFolio的分析结果
    portfolio_stats = strat.analyzers.pyfolio.get_pf_items()
    returns, positions, transactions, gross_lev = portfolio_stats
    
    # 移除时区信息
    if returns is not None:
        if hasattr(returns.index, 'tz') and returns.index.tz is not None:
            returns.index = returns.index.tz_convert(None)
    
        # 确保returns是正确的格式
        if isinstance(returns, pd.Series):
            returns = returns.copy()
            returns.index = pd.to_datetime(returns.index)
    
    # 计算性能指标
    total_return = ((final_value / capital) - 1) * 100
    num_trades = trades.get('total', {}).get('total', 0)
    
    win_trades = trades.get('won', {}).get('total', 0)
    lose_trades = trades.get('lost', {}).get('total', 0)
    win_rate = (win_trades / num_trades * 100) if num_trades > 0 else 0
    
    metrics = {
        'Total Return (%)': total_return,
        'Num Trades': num_trades,
        'Win Rate (%)': win_rate,
        'Win Trades': win_trades,
        'Lose Trades': lose_trades,
        'Final Value': final_value,
        'Parameters': params
    }
    
    print(f"  样本外评估结果: 收益={total_return:.2f}%, 交易数={num_trades}, 胜率={win_rate:.2f}%")
    
    return returns, metrics

In [31]:
def optimize_parameters(data, n_trials, n_jobs):
    """
    使用Optuna优化策略参数
    
    Args:
        data: 已经加载好的回测数据，包含正确的日期范围
        n_trials: Optuna试验次数
        n_jobs: 并行任务数
    """
    
    print(f"优化参数中...")
    
    def objective(trial):
        try:
            # 创建参数字典
            params = {}
            for param_name, param_range in CONFIG['optimization_params'].items():
                if isinstance(param_range, range):
                    params[param_name] = trial.suggest_int(
                        param_name,
                        param_range.start,
                        param_range.stop - 1,
                        param_range.step
                    )
                elif isinstance(param_range, list):
                    params[param_name] = trial.suggest_categorical(param_name, param_range)
                elif isinstance(param_range, tuple) and len(param_range) == 2:
                    params[param_name] = trial.suggest_float(
                        param_name,
                        param_range[0],
                        param_range[1]
                    )
            
            # 创建Cerebro实例
            cerebro = bt.Cerebro(
                optdatas=True,
                optreturn=True,
                runonce=True,
                preload=True
            )
            
            # 使用克隆方法获取数据的副本
            cerebro.adddata(data.clone())
            
            # 添加策略
            cerebro.addstrategy(CONFIG['strategy']['class'], **params)
            
            # 设置初始资金
            cerebro.broker.setcash(CONFIG['initial_capital'])
            
            # 设置手续费
            cerebro.broker.setcommission(commission=CONFIG['commission'])
            
            # 添加分析器
            cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
            cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
            
            # 运行回测
            results = cerebro.run()
            strat = results[0]
            
            # 计算分数
            trades = strat.analyzers.trades.get_analysis()
            total_trades = trades.get('total', {}).get('total', 0)
            
            returns = strat.analyzers.returns.get_analysis()
            total_return = returns.get('rtot', 0) * 100  # 转换为百分比
            
            # 确保足够的交易次数
            min_trades = CONFIG['optimization_settings'].get('min_trades', 10)
            trade_penalty = 1.0 if total_trades >= min_trades else (total_trades / min_trades) ** 2
            
            score = total_return * trade_penalty
            
            # 打印进度信息
            # trial_idx = trial.number
            # if trial_idx % 10 == 0:
            #     print(f"  Trial {trial_idx}/{n_trials}: 分数={score:.2f}, 收益={total_return:.2f}%, 交易={total_trades}")
            
            return score
            
        except Exception as e:
            print(f"Trial {trial.number} 出错: {e}")
            import traceback
            traceback.print_exc()
            return float('-inf')
    
    # 创建优化研究
    study = optuna.create_study(direction="maximize")
    
    # 并行运行优化
    study.optimize(
        objective,
        n_trials=n_trials,
        n_jobs=n_jobs,
        catch=(Exception,)
    )
    
    # 提取最佳参数
    best_params = study.best_params
    print(f"最佳参数: {best_params}, 分数: {study.best_value:.2f}")
    
    return best_params


In [32]:
def get_optimization_progress(reports_path):
    """
    扫描报告目录，获取各个交易对-时间周期的优化进度
    
    返回:
    dict: 包含每个交易对-时间周期的最新进度信息
    """
    progress = {}
    
    # 检查目录是否存在
    if not os.path.exists(reports_path):
        print(f"报告目录 {reports_path} 不存在")
        return progress
    
    # 扫描目录下的所有文件
    for filename in os.listdir(reports_path):
        # 检查结果CSV文件
        if filename.startswith("results_") and filename.endswith(".csv"):
            # 从文件名提取交易对和时间周期
            parts = filename.replace("results_", "").replace(".csv", "").split("_")
            if len(parts) >= 2:
                symbol = parts[0]
                timeframe = parts[1]
                key = f"{symbol}_{timeframe}"
                
                try:
                    # 读取CSV文件获取最后一个窗口信息
                    df = pd.read_csv(os.path.join(reports_path, filename))
                    if not df.empty:
                        last_row = df.iloc[-1]
                        progress[key] = {
                            'symbol': symbol,
                            'timeframe': timeframe,
                            'last_window': last_row['Window'],
                            'last_oos_end': last_row['OOS End'],
                            'last_oos_end_timestamp': pd.to_datetime(last_row['OOS End']).timestamp(),
                            'source': 'results_csv'
                        }
                except Exception as e:
                    print(f"读取文件 {filename} 时出错: {e}")
    
    # 检查参数文件和收益率文件作为备用
    for filename in os.listdir(reports_path):
        if (filename.startswith("params_") or filename.startswith("returns_")) and filename.endswith(".pkl"):
            parts = filename.replace("params_", "").replace("returns_", "").replace(".pkl", "").split("_")
            if len(parts) >= 2:
                symbol = parts[0]
                timeframe = parts[1]
                key = f"{symbol}_{timeframe}"
                
                # 如果没有在结果CSV中找到此交易对-时间周期，尝试从pickle文件获取信息
                if key not in progress:
                    try:
                        with open(os.path.join(reports_path, filename), 'rb') as f:
                            data = pickle.load(f)
                            
                        if filename.startswith("params_") and 'periods' in data:
                            last_period = data['periods'][-1]
                            progress[key] = {
                                'symbol': symbol,
                                'timeframe': timeframe,
                                'last_window': last_period.get('Window', 0),
                                'last_oos_end': last_period.get('OOS End', None),
                                'last_oos_end_timestamp': pd.to_datetime(last_period.get('OOS End')).timestamp() if last_period.get('OOS End') else None,
                                'source': 'params_pkl'
                            }
                        
                        elif filename.startswith("returns_"):
                            # 假设收益率数据帧有日期索引
                            if isinstance(data, pd.Series) and not data.empty:
                                progress[key] = {
                                    'symbol': symbol,
                                    'timeframe': timeframe,
                                    'last_date': data.index[-1],
                                    'source': 'returns_pkl'
                                }
                    except Exception as e:
                        print(f"读取文件 {filename} 时出错: {e}")
    
    return progress

def run_single_walkforward(symbol, timeframe, start_date, end_date, 
                         optimization_period, out_of_sample_period,
                         n_trials, capital, n_jobs):
    """
    对单个交易对执行遍历前移优化，支持断点续优化，并尽量使最后一个IS End接近end_date
    """
    
    # 从统一的设置中提取参数
    n_trials = CONFIG['optimization_settings']['n_trials']
    capital = CONFIG['initial_capital']
    n_jobs = CONFIG['optimization_settings']['n_jobs']

    # 遍历前移窗口的数据列表
    all_periods = [] 
    all_params = []
    all_metrics = []
    all_returns = pd.Series(dtype=float)
    
    # 准备保存文件路径
    reports_path = CONFIG['reports_path']
    results_path = os.path.join(reports_path, f"results_{symbol}_{timeframe}.csv")
    equity_path = os.path.join(CONFIG['equity_curves_dir'], f"equity_{symbol}_{timeframe}.csv")
    params_path = os.path.join(reports_path, f"params_{symbol}_{timeframe}.pkl")
    returns_path = os.path.join(reports_path, f"returns_{symbol}_{timeframe}.pkl")
    
    # 确保目录存在
    os.makedirs(os.path.dirname(results_path), exist_ok=True)
    os.makedirs(os.path.dirname(equity_path), exist_ok=True)
    
    # 检查现有进度
    progress = get_optimization_progress(reports_path)
    current_key = f"{symbol}_{timeframe}"
    window_count = 0
    current_start = start_date
    
    # 加载之前的优化参数和收益率数据(如果存在)
    if os.path.exists(params_path):
        try:
            with open(params_path, 'rb') as f:
                saved_data = pickle.load(f)
                all_periods = saved_data.get('periods', [])
                all_params = saved_data.get('params', [])
                all_metrics = saved_data.get('metrics', [])
                
                if all_periods:
                    window_count = all_periods[-1].get('Window', 0)
                    print(f"从保存的参数文件加载了 {window_count} 个之前的窗口")
        except Exception as e:
            print(f"加载参数文件时出错: {e}")
    
    if os.path.exists(returns_path):
        try:
            with open(returns_path, 'rb') as f:
                all_returns = pickle.load(f)
                print(f"从文件加载了之前的收益率数据，包含 {len(all_returns)} 个观测值")
        except Exception as e:
            print(f"加载收益率文件时出错: {e}")
    
    # 确定起始日期（从上次优化结束的地方继续）
    if current_key in progress:
        progress_info = progress[current_key]
        last_oos_end = progress_info.get('last_oos_end_timestamp')
        
        if last_oos_end:
            current_start = last_oos_end
            print(f"\n发现 {symbol}-{timeframe} 已有优化记录:")
            print(f"上次完成的窗口: #{progress_info.get('last_window')}")
            print(f"上次样本外结束日期: {datetime.fromtimestamp(current_start)}")
            print(f"将从该日期继续优化\n")
    
    print(f"开始 {symbol}-{timeframe} 的遍历前移优化")
    print(f"起始日期: {datetime.fromtimestamp(current_start)}")
    print(f"结束日期: {datetime.fromtimestamp(end_date)}")
    
    # 修改循环逻辑，优先考虑最后一个窗口接近end_date
    while current_start <= end_date:
        # 计算当前窗口的优化区间和样本外区间
        is_start = current_start - optimization_period  # in-sample开始时间
        is_end = current_start  # in-sample结束时间
        oos_start = is_end  # out-of-sample开始
        
        # 检查这是否可能是最后一个窗口
        next_oos_end = oos_start + out_of_sample_period
        is_last_window = next_oos_end + out_of_sample_period > end_date
        
        # 对于最后一个窗口，调整结束时间以尽量接近end_date
        if is_last_window:
            # 确保最后一个窗口的OOS End不超过end_date
            oos_end = min(end_date, oos_start + out_of_sample_period)
            
            # 如果距离end_date还有较大空间，尝试调整窗口
            remaining_time = end_date - oos_end
            if remaining_time > 0 and remaining_time < out_of_sample_period:
                # 将OOS End向后移动，但不超过end_date
                adjusted_oos_end = min(end_date, oos_start + out_of_sample_period + remaining_time)
                
                # 对训练窗口也做相应调整
                oos_start = is_end = current_start + remaining_time
                oos_end = adjusted_oos_end
                is_start = is_end - optimization_period
        else:
            # 非最后窗口，使用标准计算
            oos_end = oos_start + out_of_sample_period
        
        window_count += 1
        print(f"\n处理窗口 #{window_count}: {symbol}-{timeframe}")
        print(f"训练区间: {datetime.fromtimestamp(is_start)} 到 {datetime.fromtimestamp(is_end)}")
        print(f"测试区间: {datetime.fromtimestamp(oos_start)} 到 {datetime.fromtimestamp(oos_end)}")
        
        # 检查OOS区间是否有效
        is_oos_valid = oos_start < oos_end and oos_start < end_date
        
        # 如果OOS区间无效，只记录一下
        if not is_oos_valid:
            print(f"警告: 样本外区间 [{datetime.fromtimestamp(oos_start)} - {datetime.fromtimestamp(oos_end)}] 无效或超出指定时间范围")
        
        # 日期字符串转换
        is_start_str = datetime.fromtimestamp(is_start).strftime('%Y-%m-%d')
        is_end_str = datetime.fromtimestamp(is_end).strftime('%Y-%m-%d')
        oos_start_str = datetime.fromtimestamp(oos_start).strftime('%Y-%m-%d')
        oos_end_str = datetime.fromtimestamp(oos_end).strftime('%Y-%m-%d')
        
        try:
            # 加载训练数据并优化参数
            train_data = load_and_resample_data(
                symbol=symbol,
                start_date=is_start_str,
                end_date=is_end_str,
                target_timeframe=timeframe,
                source_timeframe=CONFIG['source_timeframe'],
                data_path=CONFIG['data_path']
            )
            
            # 在训练集上优化参数
            best_params = optimize_parameters(
                data=train_data,
                n_trials=n_trials,
                n_jobs=n_jobs
            )
            
            # 初始化OOS结果变量
            returns = None
            metrics = {
                'Total Return (%)': None,
                'Num Trades': None,
                'Win Rate (%)': None,
                'Final Value': None,
                'Parameters': best_params
            }
            
            # 仅当OOS区间有效时执行测试
            if is_oos_valid:
                test_data = load_and_resample_data(
                    symbol=symbol,
                    start_date=oos_start_str,
                    end_date=oos_end_str,
                    target_timeframe=timeframe,
                    source_timeframe=CONFIG['source_timeframe'],
                    data_path=CONFIG['data_path']
                )
                
                # 在测试集上评估参数
                returns, metrics = evaluate_parameters(
                    data=test_data,
                    params=best_params,
                    capital=capital
                )
            else:
                print("跳过样本外测试，仅记录优化参数")
            
            # 记录结果
            period_info = {
                'Window': window_count,
                'IS Start': is_start_str,
                'IS End': is_end_str,
                'OOS Start': oos_start_str,
                'OOS End': oos_end_str,
                'OOS Valid': is_oos_valid
            }
            
            all_periods.append(period_info)
            all_params.append(best_params)
            all_metrics.append(metrics)
            
            # 合并收益率数据
            if returns is not None and not returns.empty:
                all_returns = pd.concat([all_returns, returns])
            
            # 立即保存当前窗口的结果到CSV（追加模式）
            window_results = pd.DataFrame({
                'Symbol': [symbol],
                'Timeframe': [timeframe],
                'Window': [window_count],
                'IS Start': [is_start_str],
                'IS End': [is_end_str],
                'OOS Start': [oos_start_str],
                'OOS End': [oos_end_str],
                'Return (%)': [metrics.get('Total Return (%)') if is_oos_valid else None],
                'Trades': [metrics.get('Num Trades') if is_oos_valid else None],
                'Win Rate (%)': [metrics.get('Win Rate (%)') if is_oos_valid else None],
                'OOS Valid': [is_oos_valid]
            })
            
            # 添加参数列
            for param_name, param_value in best_params.items():
                window_results[f'param_{param_name}'] = param_value
            
            # 以追加模式保存结果
            if os.path.exists(results_path):
                window_results.to_csv(results_path, mode='a', header=False, index=False)
            else:
                window_results.to_csv(results_path, index=False)
                
            # 保存当前的收益率数据
            with open(returns_path, 'wb') as f:
                pickle.dump(all_returns, f)
                
            # 保存累积的参数
            with open(params_path, 'wb') as f:
                pickle.dump({
                    'periods': all_periods,
                    'params': all_params,
                    'metrics': all_metrics
                }, f)
                
            print(f"已保存窗口 #{window_count} 的结果和参数")
            
        except Exception as e:
            print(f"处理窗口 #{window_count} 时出错: {e}")
            import traceback
            traceback.print_exc()
        
        # 移动到下一个窗口
        if is_oos_valid:
            # 如果是最后一个窗口，完成后退出循环
            if is_last_window:
                break
            current_start = oos_end
        else:
            # 如果OOS无效，直接跳出循环，避免无限循环
            break
    
    print(f"\n完成 {symbol}-{timeframe} 的遍历前移优化，共 {window_count} 个窗口")
    
    return all_periods, all_params, all_metrics, all_returns

In [33]:
def process_single_symbol_tf(symbol, timeframe, start_date_str, end_date_str, wf_settings):
    """
    处理单个交易对和时间周期的遍历前移优化

    Args:
        symbol (str): 交易对名称
        timeframe (str): 时间周期
        start_date_str (str): 开始日期字符串 
        end_date_str (str): 结束日期字符串
        wf_settings (dict): 遍历前移设置

    Returns:
        dict: 该交易对和时间周期的优化结果
    """


    print(f"开始对 {symbol}-{timeframe} 执行遍历前移优化...")

    # 转换日期
    start_date = time.mktime(datetime.strptime(start_date_str, "%Y-%m-%d").timetuple())
    end_date = time.mktime(datetime.strptime(end_date_str, "%Y-%m-%d").timetuple())

    # 计算时间周期
    optimization_period = timedelta(days=wf_settings['optimization_period_days']).total_seconds()
    out_of_sample_period = timedelta(days=wf_settings['out_of_sample_period_days']).total_seconds()

    # 提取其他设置
    n_trials = CONFIG['optimization_settings']['n_trials']
    capital = CONFIG['initial_capital']
    n_jobs = CONFIG['optimization_settings']['n_jobs']

    try:
        # 运行单个交易对的遍历前移优化
        periods, params, metrics, returns_data = run_single_walkforward(
            symbol, timeframe, start_date, end_date,
            optimization_period, out_of_sample_period,
            n_trials, capital, n_jobs
        )

        # 保存样本外returns数据
        returns_file = f"{CONFIG['reports_path']}/returns_{symbol}_{timeframe}.pkl"
        os.makedirs(os.path.dirname(returns_file), exist_ok=True)
        with open(returns_file, 'wb') as f:
            pickle.dump(returns_data, f)

        print(f"已保存returns数据到: {returns_file}")

        # 创建结果摘要
        result = {
            'Symbol': symbol,
            'Timeframe': timeframe,
            'Start Date': start_date_str,
            'End Date': end_date_str,
            'Periods': periods,
            'Parameters': params,
            'Metrics': metrics,
            'Returns File': returns_file
        }

        return result

    except Exception as e:
        print(f"处理 {symbol}-{timeframe} 时出错: {e}")
        import traceback
        traceback.print_exc()
        return None
    

## 10. 主程序执行示例

In [34]:
def aggregate_and_save_results(results, results_filename):
    """
    汇总所有交易对的结果并保存
    
    Args:
        results: 所有交易对的结果列表
        results_filename: 结果文件名
        
    Returns:
        DataFrame: 汇总后的结果
    """
 
    import os
    
    if not results:
        print("没有可汇总的结果")
        return None
    
    # 创建结果列表
    summary_rows = []
    
    for result in results:
        if not result:
            continue
            
        symbol = result['Symbol']
        timeframe = result['Timeframe']
        
        # 遍历每个窗口期
        for i, (period, params, metrics) in enumerate(zip(
                result['Periods'], result['Parameters'], result['Metrics'])):
            
            # 创建基本信息
            row = {
                'Symbol': symbol,
                'Timeframe': timeframe,
                'Window': period['Window'],
                'IS Start': period['IS Start'],
                'IS End': period['IS End'],
                'OOS Start': period['OOS Start'],
                'OOS End': period['OOS End'],
                'Return (%)': metrics['Total Return (%)'],
                'Trades': metrics['Num Trades'],
                'Win Rate (%)': metrics['Win Rate (%)'],
                'ReturnsFile': result['Returns File']
            }
            
            # 添加参数信息
            for param_name, param_value in params.items():
                row[f'param_{param_name}'] = param_value
                
            summary_rows.append(row)
    
    # 创建DataFrame
    df = pd.DataFrame(summary_rows)
    
    # 保存结果
    results_path = os.path.join(CONFIG['reports_path'], results_filename)
    df.to_csv(results_path, index=False)
    print(f"已保存汇总结果到: {results_path}")
    
    return df


In [35]:
def batch_optimize_walk_forward():
    """
    多交易对并行遍历前移优化的主函数
    """
    import time
    import os
    import threading
    from queue import Queue
    from datetime import datetime, timedelta

    print(f"\n{'#'*80}")
    print(f"启动MeanReverter WalkForward多交易对并行优化框架")
    print(f"当前时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'#'*80}\n")
    
    # 解析数据路径并更新CONFIG
    resolved_data_path = resolve_data_path()
    CONFIG['resolved_data_path'] = resolved_data_path
    CONFIG['data_path'] = resolved_data_path
    
    print(f"使用数据路径: {resolved_data_path}")
    
    # 获取交易对列表
    all_symbols = CONFIG['selected_symbols']
    valid_symbols = []
    
    # 检查交易对数据可用性...
    print("检查交易对数据可用性...")
    for symbol in all_symbols:
        try:
            # 使用解析后的数据路径
            first_day = CONFIG['start_date']
            file_path = os.path.join(
                resolved_data_path, 
                first_day,
                f"{first_day}_{symbol}_USDT_1m.csv"
            )
            
            if os.path.exists(file_path):
                valid_symbols.append(symbol)
                print(f"√ {symbol} - 数据可用")
            else:
                print(f"× {symbol} - 未找到数据文件: {file_path}")
        except Exception as e:
            print(f"× {symbol} - 检查数据出错: {e}")
    
    print(f"共找到 {len(valid_symbols)}/{len(all_symbols)} 个有效交易对")
    
    # 获取目标时间周期
    timeframes = CONFIG['target_timeframes']
    print(f"将对以下 {len(valid_symbols)} 个有效交易对进行遍历前移优化: {valid_symbols}")
    
    # 创建结果目录
    os.makedirs(CONFIG['reports_path'], exist_ok=True)
    os.makedirs(CONFIG['equity_curves_dir'], exist_ok=True)
    
    # 使用有效的交易对列表创建任务队列
    task_queue = Queue()
    for symbol in valid_symbols:
        for tf in timeframes:
            task_queue.put((symbol, tf))
    
    # 存储所有结果的列表
    all_results = []
    results_lock = threading.Lock()
    
    # 计算日期
    start_date_str = CONFIG['start_date']
    end_date_str = CONFIG['end_date']
    
    # 获取WalkForward设置
    wf_settings = CONFIG['walkforward_settings']
    optimization_period = timedelta(days=wf_settings['optimization_period_days']).total_seconds()
    out_of_sample_period = timedelta(days=wf_settings['out_of_sample_period_days']).total_seconds()
    
    # 从优化设置中获取通用参数
    capital = CONFIG['initial_capital']
    n_threads = wf_settings['n_threads']  # 多交易对处理的线程数
    
    def worker():
        """工作线程函数，处理队列中的任务"""
        while not task_queue.empty():
            try:
                symbol, tf = task_queue.get(block=False)
                print(f"开始处理交易对-时间周期: {symbol}-{tf}")
                
                # 处理单个交易对和时间周期
                result = process_single_symbol_tf(symbol, tf, 
                                                  start_date_str, end_date_str, 
                                                  wf_settings)
                
                # 安全地添加结果
                if result:
                    with results_lock:
                        all_results.append(result)
                
                task_queue.task_done()
                print(f"完成处理交易对-时间周期: {symbol}-{tf}")
            except Exception as e:
                print(f"处理 {symbol}-{tf} 时出错: {e}")
                import traceback
                traceback.print_exc()
                try:
                    task_queue.task_done()
                except:
                    pass
    
    # 创建并启动线程
    start_time = time.time()
    # 线程数的设置
    n_threads = min(CONFIG['walkforward_settings'].get('n_threads', 10), 
                    len(valid_symbols) * len(timeframes))
    threads = []
    
    print(f"启动 {n_threads} 个工作线程...")
    for _ in range(n_threads):
        t = threading.Thread(target=worker)
        t.daemon = True
        threads.append(t)
        t.start()
    
    # 等待所有线程完成
    for t in threads:
        t.join()
    
    # 生成汇总报告
    results_filename = CONFIG['results_filename_template'].format(
        strategy_name=CONFIG['strategy']['name'],
        start_date=start_date_str.replace('-', ''),
        end_date=end_date_str.replace('-', '')
    )
    
    final_results = aggregate_and_save_results(all_results, results_filename)
    
    end_time = time.time()
    
    print(f"\n{'#'*80}")
    print(f"WalkForward多交易对并行优化框架执行完成")
    print(f"处理了 {len(valid_symbols)} 个交易对, {len(timeframes)} 个时间周期")
    print(f"开始时间: {datetime.fromtimestamp(start_time).strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"结束时间: {datetime.fromtimestamp(end_time).strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"总耗时: {(end_time - start_time):.2f}秒 ({(end_time - start_time)/3600:.2f}小时)")
    print(f"{'#'*80}\n")
    
    return all_results, final_results

In [36]:
if __name__ == '__main__':
    # 使用多交易对批处理模式
    all_results, final_results = batch_optimize_walk_forward()


################################################################################
启动MeanReverter WalkForward多交易对并行优化框架
当前时间: 2025-04-05 19:36:58
################################################################################

找到有效数据路径: .\..\..\futures
使用数据路径: .\..\..\futures
检查交易对数据可用性...
√ AAVEUSDT - 数据可用
√ 1000PEPEUSDT - 数据可用
√ APTUSDT - 数据可用
共找到 3/3 个有效交易对
将对以下 3 个有效交易对进行遍历前移优化: ['AAVEUSDT', '1000PEPEUSDT', 'APTUSDT']
启动 3 个工作线程...
开始处理交易对-时间周期: AAVEUSDT-1H
开始对 AAVEUSDT-1H 执行遍历前移优化...
开始处理交易对-时间周期: 1000PEPEUSDT-1H
开始对 1000PEPEUSDT-1H 执行遍历前移优化...
开始处理交易对-时间周期: APTUSDT-1H
开始对 APTUSDT-1H 执行遍历前移优化...
读取文件 results_1000PEPEUSDT_1H.csv 时出错: Error tokenizing data. C error: Expected 25 fields in line 28, saw 26

读取文件 results_1000PEPEUSDT_1H.csv 时出错: Error tokenizing data. C error: Expected 25 fields in line 28, saw 26

读取文件 results_1000PEPEUSDT_1H.csv 时出错: Error tokenizing data. C error: Expected 25 fields in line 28, saw 26

读取文件 results_AAVEUSDT_1H.csv 时出错: Error tokenizing data. C error:

[I 2025-04-05 19:36:59,120] A new study created in memory with name: no-name-046da589-c813-4ceb-b15f-fe2ccef25e7f


优化参数中...


[I 2025-04-05 19:36:59,742] A new study created in memory with name: no-name-2646ceb0-8390-4f4a-96c6-24c6d7672fa8
[I 2025-04-05 19:37:00,180] A new study created in memory with name: no-name-374a94c8-ea59-4d3f-a6b3-3e9b354dadf3


优化参数中...
优化参数中...


...

在样本外区间评估参数...
  样本外评估结果: 收益=-2.97%, 交易数=3, 胜率=66.67%
已保存窗口 #32 的结果和参数

完成 AAVEUSDT-1H 的遍历前移优化，共 32 个窗口
已保存returns数据到: reports_walkforward/returns_AAVEUSDT_1H.pkl
完成处理交易对-时间周期: AAVEUSDT-1H


[I 2025-04-05 20:00:04,402] Trial 164 finished with value: -2.2619165259185365 and parameters: {'port': 12, 'ps2': 3, 'ps3': 3, 'ps4': 7, 'ps5': 13, 'ps6': 12, 'ps7': 31, 'ps8': 45, 'ma_length': 180, 'rsi_length': 10, 'oversold': 23, 'profit_target_percent': 5, 'profit_target_percent_all': 5, 'take_profit_progression': 11, 'entry_on': True}. Best is trial 129 with value: 2.439889924816992.
[I 2025-04-05 20:00:04,595] Trial 174 finished with value: -5.178080660047475 and parameters: {'port': 18, 'ps2': 2, 'ps3': 2, 'ps4': 4, 'ps5': 8, 'ps6': 17, 'ps7': 27, 'ps8': 31, 'ma_length': 150, 'rsi_length': 18, 'oversold': 30, 'profit_target_percent': 5, 'profit_target_percent_all': 3, 'take_profit_progression': 9, 'entry_on': False}. Best is trial 80 with value: 21.51719778940558.
[I 2025-04-05 20:00:05,052] Trial 177 finished with value: 13.081056781904504 and parameters: {'port': 18, 'ps2': 3, 'ps3': 2, 'ps4': 4, 'ps5': 8, 'ps6': 17, 'ps7': 27, 'ps8': 31, 'ma_length': 150, 'rsi_length': 19, '

最佳参数: {'port': 18, 'ps2': 2, 'ps3': 2, 'ps4': 5, 'ps5': 6, 'ps6': 18, 'ps7': 30, 'ps8': 31, 'ma_length': 180, 'rsi_length': 19, 'oversold': 29, 'profit_target_percent': 5, 'profit_target_percent_all': 3, 'take_profit_progression': 9, 'entry_on': False}, 分数: 21.52


[I 2025-04-05 20:00:29,522] Trial 191 finished with value: 19.56645358097945 and parameters: {'port': 17, 'ps2': 4, 'ps3': 5, 'ps4': 6, 'ps5': 10, 'ps6': 17, 'ps7': 24, 'ps8': 49, 'ma_length': 90, 'rsi_length': 19, 'oversold': 37, 'profit_target_percent': 4, 'profit_target_percent_all': 3, 'take_profit_progression': 7, 'entry_on': True}. Best is trial 191 with value: 19.56645358097945.
[I 2025-04-05 20:00:29,647] Trial 189 finished with value: 0.27714205125682173 and parameters: {'port': 17, 'ps2': 4, 'ps3': 5, 'ps4': 6, 'ps5': 10, 'ps6': 17, 'ps7': 24, 'ps8': 49, 'ma_length': 90, 'rsi_length': 17, 'oversold': 23, 'profit_target_percent': 4, 'profit_target_percent_all': 3, 'take_profit_progression': 7, 'entry_on': True}. Best is trial 191 with value: 19.56645358097945.
[I 2025-04-05 20:00:30,356] Trial 193 finished with value: 0.06284369325989937 and parameters: {'port': 17, 'ps2': 4, 'ps3': 5, 'ps4': 6, 'ps5': 10, 'ps6': 17, 'ps7': 24, 'ps8': 33, 'ma_length': 140, 'rsi_length': 19, 'o

最佳参数: {'port': 17, 'ps2': 4, 'ps3': 5, 'ps4': 6, 'ps5': 10, 'ps6': 17, 'ps7': 24, 'ps8': 49, 'ma_length': 90, 'rsi_length': 19, 'oversold': 37, 'profit_target_percent': 4, 'profit_target_percent_all': 3, 'take_profit_progression': 7, 'entry_on': True}, 分数: 19.57
在样本外区间评估参数...
  样本外评估结果: 收益=-0.85%, 交易数=1, 胜率=0.00%
已保存窗口 #32 的结果和参数

完成 1000PEPEUSDT-1H 的遍历前移优化，共 32 个窗口
已保存returns数据到: reports_walkforward/returns_1000PEPEUSDT_1H.pkl
完成处理交易对-时间周期: 1000PEPEUSDT-1H
在样本外区间评估参数...
  样本外评估结果: 收益=-1.42%, 交易数=1, 胜率=0.00%
已保存窗口 #32 的结果和参数

完成 APTUSDT-1H 的遍历前移优化，共 32 个窗口
已保存returns数据到: reports_walkforward/returns_APTUSDT_1H.pkl
完成处理交易对-时间周期: APTUSDT-1H
已保存汇总结果到: reports_walkforward\walkforward_results_RSIPAPTP_20240101-20250403.csv

################################################################################
WalkForward多交易对并行优化框架执行完成
处理了 3 个交易对, 1 个时间周期
开始时间: 2025-04-05 19:36:58
结束时间: 2025-04-05 20:00:33
总耗时: 1415.70秒 (0.39小时)
######################################################################